In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data Visualization 
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import warnings 
warnings.filterwarnings('ignore')
# Any results you write to the current directory are saved as output.

In [ ]:
# Combined Data set, train and test data sets
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
datasets = [train, test]

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train.dtypes

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
plt.subplots(figsize=(12,9))
cmap = sns.diverging_palette(250, 10, as_cmap=True)
sns.heatmap(train.drop(["PassengerId"], axis = 1).corr(), annot = True, cmap = cmap)

In [ ]:
for dataset in datasets:
    dataset.drop(["Cabin"], axis = 1, inplace = True)

In [ ]:
train["Embarked"].value_counts()

In [ ]:
for dataset in datasets:
    dataset["Embarked"].fillna("S", inplace = True)

In [ ]:
train[["Sex", "Survived"]].groupby(["Sex"], as_index = False).mean().sort_values(by="Survived", ascending = False)

In [ ]:
train[["Pclass", "Survived"]].groupby(["Pclass"], as_index = False).mean().sort_values(by="Pclass")

In [ ]:
train[["Embarked", "Survived"]].groupby(["Embarked"], as_index = False).mean().sort_values(by = "Embarked")

In [ ]:
pd.crosstab([train["Embarked"], train["Pclass"]], [train["Sex"], train["Survived"]], margins = True).style.background_gradient(cmap = cmap)

In [ ]:
for dataset in datasets:
    dataset["Embarked"] = dataset["Embarked"].map({"C": 0, "Q": 1, "S": 2})
train.head()

In [ ]:
for dataset in datasets:
    dataset["FamilySize"] = dataset["SibSp"]+dataset["Parch"]+1
train.head()

In [ ]:
pd.crosstab(train["FamilySize"], train["Survived"], margins = True).style.background_gradient(cmap = cmap)

In [ ]:
train.head()

In [ ]:
for dataset in datasets:
    dataset.drop(["Ticket"], axis = 1, inplace = True)
train.head()

In [ ]:
for dataset in datasets:
    dataset["Sex"] = dataset["Sex"].map({"male":0, "female":1})
train.head()

In [ ]:
for dataset in datasets:
    dataset["Title"] = dataset["Name"].str.extract(' ([A-Za-z]+)\.', expand=False)
train.head()

In [ ]:
pd.crosstab(train["Title"], train["Sex"], margins = True).sort_values(by = "All", ascending = False)

In [ ]:
for dataset in datasets:
    dataset["Title"] = dataset["Title"].replace(["Dr", "Rev", "Major", "Col", "Mlle", "Don", "Jonkheer", "Lady", "Mme", "Countess", "Ms", "Sir", "Capt"], "Other")
    dataset["Title"] = dataset["Title"].map({"Mr":0, "Miss":1, "Mrs":2 , "Master": 3, "Other" :4})

In [ ]:
for dataset in datasets:
    dataset["Title"].fillna(0, inplace=True)

In [ ]:
for dataset in datasets:
     dataset.drop(["Name"], axis = 1, inplace=True)

In [ ]:
dataset.head()

In [ ]:
for dataset in datasets:
    dataset["Age"].fillna(dataset["Age"].mean(), inplace = True)
    

In [ ]:
for dataset in datasets:
    dataset["Fare"].fillna(dataset["Fare"].mean(), inplace = True)

In [ ]:
train.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(train.drop(["Survived"], axis = 1), train["Survived"])

In [ ]:
X_train.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=100,
                                criterion='gini',
                                max_depth=5,
                                min_samples_split=10,
                                min_samples_leaf=5,
                                random_state=0)
classifier.fit(X_train, Y_train)

print("Random Forest score: {0:.2}".format(classifier.score(X_test, Y_test)))

In [ ]:
prediction = classifier.predict(test)

submission = pd.DataFrame({"PassengerId": test["PassengerId"], "Survived": prediction})
submission.to_csv("submission.csv", index=False)